<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark_join_multiple_dstreams_json2dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [2]:
%%capture
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [35]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

In [36]:
kafka_topic_name = "my-first-topic"
kafka_topic_name2 = "my-first-topic2"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [72]:
sc = pyspark.SparkContext()
spark = SparkSession(sc)
ssc = StreamingContext(sc,2) # Show results every 2 seconds

In [73]:
def readMyStream(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)
    print(df.dtypes)
    print('Started the Process')
    print('Selection of Columns')
    df = df.select('firstname','lastname') #.where(col("timestamp").isNotNull())
    df.show()

In [74]:
kvs1 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'smallest'
                        })

kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })
#,
#                        valueDecoder=decoder)

kvs2.pprint()
parsed = kvs2.map(lambda x: x[1])
parsed.foreachRDD( lambda rdd: readMyStream(rdd) )


#pairs = lines.map(lambda x: (x.split(" ")[0], x))
'''
kvs1.pprint()
k1 = kvs1.map(lambda x:x[1]).flatMap(lambda x: x.split(",")).map(lambda x: (x.split(":")[0],x.split(":")[1]))
k1.pprint(30)

k2 = kvs2.map(lambda x:x[1]).flatMap(lambda x: x.split(",")).map(lambda x: (x.split(":")[0],x.split(":")[1]))
k2.pprint(30)

k1.fullOuterJoin(k2).pprint(30)
'''
#wc = kvs2.map(lambda x: x[1]).flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
#wc.pprint()
#kvs3 = wc.join(kvs1)
#kvs3.pprint()
'''kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group1',
                        'auto.offset.reset':'smallest'
                        })

kvs2.pprint()'''

'''step0 = kvs.map(lambda x: x[1])
#print("step0") # print once
step0.pprint()
step1 = step0.flatMap(lambda line: line.split(' '))
step1.pprint()
step2 = step1.map(lambda word: (word, 1))
step2.pprint()
step3 = step2.reduceByKey(lambda a, b: a+b) #.filter(lambda y: y[1]>5)
step3.pprint()'''

ssc.start()
ssc.awaitTerminationOrTimeout(30)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2022-10-31 09:30:22
-------------------------------------------

-------------------------------------------
Time: 2022-10-31 09:30:24
-------------------------------------------

-------------------------------------------
Time: 2022-10-31 09:30:26
-------------------------------------------

-------------------------------------------
Time: 2022-10-31 09:30:28
-------------------------------------------

-------------------------------------------
Time: 2022-10-31 09:30:30
-------------------------------------------
('8856d3a2-585d-4219-a581-0ffb23998a41', ' {"id":2,"firstname":"Peter2","middlename":"","lastname":"Parker3","dob_year":2018,"dob_month":1,"gender":"M","salary":3000} ')

Started the Process
Selection of Columns
+---------+--------+
|firstname|lastname|
+---------+--------+
|   Peter2| Parker3|
+---------+--------+

-------------------------------------------
Time: 2022-10-31 09:30:32
--------------------------------------

In [71]:
ssc.stop()
sc.stop()